I'm using 3DDFA_V2 to add white mask on face<br>
credit:
[3DDFA_V2](https://github.com/cleardusk/3DDFA_V2)

In [ ]:
# @title ##**Mount drive and Note your video id** { display-mode: "form" }

from google.colab import drive
from IPython.display import clear_output
import os
%cd /content/
drive.mount('/content/gdrive')

!mkdir "/content/export_each_frame"
!mkdir "/content/face_mask_frame"
video_folder="/content/gdrive/MyDrive/FACE_MASK_INPUT/"
save_path="/content/gdrive/MyDrive/FACE_MASK_OUTPUT/"
make_folder=["FACE_MASK_INPUT","FACE_MASK_OUTPUT"]
for i in make_folder:
  try:
    os.mkdir("/content/gdrive/MyDrive/"+i)
  except:
    pass
clear_output()  
print("Upload video in this folder: /content/gdrive/MyDrive/FACE_MASK/\n\n")

video_id=1
table={}
ids=[]
videos=[]
id_monitor={}
for i in os.listdir(video_folder):
  ids.append(video_id)
  videos.append(i)
  id_monitor[video_id]=i
  video_id=video_id+1
table["video_name"]=videos
table["video_id"]=ids  
import pandas as pd
df = pd.DataFrame(table)
df

Upload video in this folder: /content/gdrive/MyDrive/FACE_MASK/




,video_name,video_id
0,WIN_20211107_19_54_50_Pro.mp4,1


In [ ]:
# @title ##**Type the video id from the table** { display-mode: "form" }

%cd /content
import cv2

video_id = 1  #@param {type: "number"}
try:
  video_path=video_folder+id_monitor[int(video_id)]
  cap=cv2.VideoCapture(video_path)
  video_fps = cap.get(cv2.CAP_PROP_FPS)
  video_fps=round(video_fps)
  os.chdir("/content/export_each_frame")
  get_files=os.listdir()
  for i in get_files:
    try:
      os.remove(i)
    except:
      pass
      # -vf fps={video_fps}/1
  var_1=os.system(f"ffmpeg -i {video_path}  %06d.png")
  os.chdir("/content")
  clear_output()
  if var_1==0:
    print("Extract Frame Successfully")
  else:
    print("Frame Extract Failed")
except Exception as e :
  print("Enter currect video id"+str(e))    

Extract Frame Successfully


In [ ]:
# if not clone, clone it
# @title ##**Run our model** { display-mode: "form" }
%rm -rf /content/3DDFA_V2
%cd /content
!git clone https://github.com/cleardusk/3DDFA_V2.git
%cd 3DDFA_V2
clear_output()
!sh ./build.sh
clear_output()
# before import, make sure FaceBoxes and Sim3DR are built successfully, e.g.,

import cv2
import yaml

from FaceBoxes import FaceBoxes
from TDDFA import TDDFA
from utils.render import render
from utils.depth import depth
from utils.pncc import pncc
from utils.uv import uv_tex
from utils.pose import viz_pose
from utils.serialization import ser_to_ply, ser_to_obj
from utils.functions import draw_landmarks, get_suffix

import matplotlib.pyplot as plt
from skimage import io
clear_output()
# load config
cfg = yaml.load(open('configs/mb1_120x120.yml'), Loader=yaml.SafeLoader)

# Init FaceBoxes and TDDFA, recommend using onnx flag
onnx_flag = True  # or True to use ONNX to speed up
if onnx_flag:
    !pip install onnxruntime
    
    import os
    os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
    os.environ['OMP_NUM_THREADS'] = '4'
    from FaceBoxes.FaceBoxes_ONNX import FaceBoxes_ONNX
    from TDDFA_ONNX import TDDFA_ONNX

    face_boxes = FaceBoxes_ONNX()
    tddfa = TDDFA_ONNX(**cfg)
else:
    face_boxes = FaceBoxes()
    tddfa = TDDFA(gpu_mode=False, **cfg)
clear_output()    

from PIL import Image
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

def face_filter(image_path):
  img_url = image_path
  img = io.imread(img_url)
  img = img[..., ::-1]  # RGB -> BGR
  boxes = face_boxes(img)
  param_lst, roi_box_lst = tddfa(img, boxes)
  dense_flag=True
  ver_lst = tddfa.recon_vers(param_lst, roi_box_lst, dense_flag=dense_flag)
  overlap=depth(img, ver_lst, tddfa.tri, show_flag=False)
  open_cv_image = np.array(overlap) 
  file_name=f"/content/face_mask_frame/{image_path.split('/')[-1]}"
  cv2.imwrite(file_name,open_cv_image)



In [ ]:
# @title ##**put mask on image** { display-mode: "form" }
import glob
%cd /content/3DDFA_V2
clear_output()  
for k in os.listdir("/content/face_mask_frame/"):
  try:
    os.remove(f"/content/face_mask_frame/{k}")
  except:
    pass

for i in glob.glob("/content/export_each_frame/*png"):
  try:
    face_filter(i)
  except:
    print(f"{i.split('/')[-1]} create an error")
print("Face mask added")
  

Face mask added


In [ ]:
# @title ##**Making the video** { display-mode: "form" }

import glob
import cv2
os.chdir("/content/face_mask_frame")
new_id_list=[]
images=glob.glob("*png")
image_dict={}
for i in images:
  sort_id=int(i.split(".png")[0])
  image_dict[sort_id]=i
  new_id_list.append(sort_id)
image_name_id=sorted(new_id_list)

images=glob.glob("/content/face_mask_frame/*png")
video_name= 'white_mask.mp4' 
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
frame = cv2.imread(images[0])
height, width, layers = frame.shape
video = cv2.VideoWriter(video_name, fourcc, int(video_fps), (width, height))
for i in range(len(image_name_id)):
    image_path=f"{image_dict[image_name_id[i]]}"
    video.write(cv2.imread(image_path))
video.release()
var4=os.system(f"ffmpeg -i {video_path} audio.wav -y")
if var4==0:
    print("Successfully export audio")
else:
    print("Failed to export audio")
export_file_name=f'{save_path}face_mask_{video_path.split("/")[-1]}'
var5=os.system(f"ffmpeg -i {video_name} -i audio.wav -c:v copy -c:a aac -map 0:v:0 -map 1:a:0 {export_file_name} -y")
if var5==0:
    print(f"Video save at {export_file_name}")
else:
    print("Failed to replace audio in output file")    
os.chdir("/content/")

from google.colab import files
files.download(export_file_name)